<a href="https://colab.research.google.com/github/Nathan-Nicolau/RedesNeuraisTCC/blob/main/RedeConvolucionalTCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 **Sem nenhuma execução prévia ainda**

---



In [ ]:
!pip install rasterio

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pylab as rcParams
import matplotlib

matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 14
matplotlib.rcParams['ytick.labelsize'] = 12
matplotlib.rcParams['text.color'] = 'k'
import seaborn as sns
plt.style.use('fivethirtyeight')
rcParams['figure.figsize'] = 18, 6
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

caminho no Drive das imagens do dataset

In [ ]:
path = 'drive/...caminho'
classList = os.listdir(path)
#verificação da quantidade de classes
print(classList)

In [ ]:
x = [] #Vetor que receberá as imaagens do Dataset
y = [] #Vetor que receberá as informações das classes
i = 0

for i in range(len(classList)):
  pathClass = os.path.join(path,classList[i])
  imgList = os.listdir(pathClass)
  for j in range(len(imgList)):
    pathImg = ps.path.join(pathClass,imgList[j])
    src = rasterio.open(pathImg)
    im = src.read()
    im = im.transpose([1,2,0])
    im1 = im[:,:,1:9]
    im2 = im[:,:,11:]
    im = np.dstack((im1,im2))
    print(str(j) + ' ' +classList[i])
    y.append(i)
    x.append(im)

In [ ]:
#Visualização do "shape" dos dados
x = np.array(x)
y = np.array(y)

In [ ]:
#Uma forma de verificação opicional da razão de quantidade de imagens
#pela quantidade de classes existentes

classe, qtd = np.unique(y,return_counts=True)
df = pd.DataFrame(data=qtd,index=classList, columns=['Amostras'])
ax = sns.barplot(x = df.index, y="Amostras", data=df)

In [ ]:
#Importação das classes/bibliotecas para desenvolvimento da Rede Convolucional
from keras.models import Model, Sequential
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten,Dense,BatchNormalization, Dropout, Activation
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from keras.activations import relu
from keras.losses import binary_crossentropy
from keras import backend as K
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils

In [ ]:
#conversão do vetor "y" (valores das classes) para uma representação categórica com valores entre 0 e 1
y = np_utils.to_categorical(y)

#Separação de dados entre: Treinamento e Testes
xTrain, xTest, yTrain, yTest = train_test_split(x,y,test_size=0.3,random_state=10)

In [ ]:
print('Dados x de Treinamento: ' + xTrain.shape)
print('Dados x de Testes' + yTest.shape)

print('Dados y de Treinamento: ' + yTrain.shape)
print('Dados y de Testes: ' + yTest.shape)

**Analisar necessidade: ** Como estamos usando imagens do Sentinel 2, precisamos dividir os valores de pixel brutos para colocar em um intervalo entre 0 e 1. Primeiro com o conjunto de teste e depois com o conjunto de treinamento:

In [ ]:
xTest = xTest/10000.0
xTrain = xTrain/10000.0

xTrain.shape[1:]

In [ ]:
#Rede convolucional montado com o Keras
Rede = Sequential()

Rede.add(Conv2D(32,(3,3),kernel_initializer="he_normal",padding="same",input_shape=(xTrain.shape[1:])))
Rede.add(Activation('relu'))
Rede.add(MaxPooling2D(pool_size=(2,2),strides=2))
Rede.add(Conv2D(64,(3,3),kernel_initializer='he_normal',padding='same'))
Rede.add(Activation('relu'))
Rede.add(MaxPooling2D(pool_size=(2,2),strides=2))
Rede.add(Conv2D(128,(3,3),kernel_initializer='he_normal',padding='same'))
Rede.add(Activation('relu'))
Rede.add(MaxPooling2D(pool_size=(2,2),strides=2))
Rede.add(Conv2D(256,(3,3),kernel_initializer='he_normal',padding='same'))
Rede.add(Activation('relu'))
Rede.add(MaxPooling2D(pool_size=(2,2),strides=2))
Rede.add(Flatten())
Rede.add(Dropout(0.5))
Rede.add(Dense(512))
Rede.add(Activation('relu'))
Rede.add(Dense(10))
Rede.add(Activation('softmax'))
Rede.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.00001, decay=1e-6), metrics=['accuracy'])
Rede.summary()

#Treinamento da Rede com os dados já pode ser feita


In [ ]:
#Parâmetros podem ser mudados
history = Rede.fit(x = xTrain, y=yTrain, batch_size=64, epochs=200, verbose=1,shuffle=True, validation_split=0.2)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Resultados de precisão da RNC')
plt.ylabel('Precisão')
plt.xlabel('Épocas')
plt.legend(['Dados de treinamento', 'Dados de validação'], loc='lower right')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Resultados de perda da RNC')
plt.ylabel('Perda')
plt.xlabel('Épocas]')
plt.legend(['Daoos de treinamento', 'Dados de testes'], loc='upper right')
plt.show()

Predição das classes das imagens do dataset que foram separadas para testes

In [ ]:
predict = Rede.predict(xTest)
pred = np.argmax(predict,axis=1)
true = np.argmax(yTest, axis=1)

#Cálculo da precisão global dos dados de teste
accuracy = accuracy_score(true,pred)
print('Precisão Global:(porcentagem)' + accuracy * 100 + "%")

#outra forma de classificação de precisões obtidas com o classification_report

print(classification_report(true,pred))